In [ ]:
from var4d_components import Var4D_Components
from visualize_results import Visualize_Obs, Visualize_Fluxes, Diagnostic_Plots

# Balancing data and prior uncertainty
Surface flux inversions are Bayesian estimates that trade off between our knowledge of fluxes prior to making atmospheric observations and our knowledge of the atmospheric state as reflected in observations. The former is represented by $\mathbf{S_0}^{-1}$ (smaller the uncertainty, "better" the knowledge) and the latter is represented by $\mathbf{S_z}^{-1}$. In this exercise, we will vary the balance between the two and see what the solution looks like.
## Set up an inversion to assimilate only NOAA surface observatories
Let's assimilate just the four NOAA observatories at Barrow, Mauna Loa, Samoa, and South Pole.

In [ ]:
var4d = Var4D_Components('only_noaa_observatories', verbose=False, store_intermediate=True) # change verbose to False to see fancy progress bars and suppress prints
flux_corr_structure = {'temp_corr': 2.0} # 2-month temporal correlation, no horizontal correlation
obs_assim_dict = {'sites': ['mlo', 'spo', 'brw', 'smo']} # just four observatories

First, do the base case of NEE uncertainty = 0.25 $\times$ abs(NEE) and ocean flux uncertainty = 0.5 $\times$ abs(ocean flux)

In [ ]:
prior_flux_unc_dict = {'prior_unc_source': 'nee', 'prior_unc_scale': {'land': 0.25, 'ocean': 0.5}}
var4d.var4d_setup(obs_to_assim=obs_assim_dict, corr_structure=flux_corr_structure, **prior_flux_unc_dict)
var4d.var4d_chain(gradnorm=1.0E-5)

How well did we fit the obs, assimilated and un-assimilated?

In [ ]:
po1 = Visualize_Obs('only_noaa_observatories')
po1.plot_site(['spo','mlo','smo', 'brw'])

How well did we reproduce the __true__ fluxes, which is the end goal?

In [ ]:
vf1 = Visualize_Fluxes('only_noaa_observatories')
vf1.plot_region(['Globe', 'North American Boreal', 'North American Temperate', 'South American Tropical'])

Now let's __tighten__ the prior flux uncertainty by a factor of __100__. **What do you expect?**

In [ ]:
var4d = Var4D_Components('explore_unc_balance', verbose=False, store_intermediate=True) # change verbose to False to see fancy progress bars and suppress prints
flux_corr_structure = {'temp_corr': 2.0} # 2-month temporal correlation, no horizontal correlation
prior_flux_unc_dict = {'prior_unc_source': 'nee', 'prior_unc_scale': {'land': 0.0025, 'ocean': 0.005}}
var4d.var4d_setup(obs_to_assim=obs_assim_dict, corr_structure=flux_corr_structure, **prior_flux_unc_dict)
var4d.var4d_chain(gradnorm=1.0E-5)

In [ ]:
po2 = Visualize_Obs('explore_unc_balance')
po2.plot_site(['spo','mlo','smo', 'brw'])

In [ ]:
vf2 = Visualize_Fluxes('explore_unc_balance')
vf2.plot_region(['Globe', 'North American Boreal', 'North American Temperate', 'South American Tropical'])

What if we __loosen__ prior flux uncertainty by a factor of __100__?

In [ ]:
prior_flux_unc_dict = {'prior_unc_source': 'nee', 'prior_unc_scale': {'land': 25.0, 'ocean': 50.0}}
var4d.var4d_setup(obs_to_assim=obs_assim_dict, corr_structure=flux_corr_structure, **prior_flux_unc_dict)
var4d.var4d_chain(gradnorm=1.0E-5)

In [ ]:
po3 = Visualize_Obs('explore_unc_balance')
po3.plot_site(['spo','mlo','smo', 'brw'])

In [ ]:
vf3 = Visualize_Fluxes('explore_unc_balance')
vf3.plot_region(['Globe', 'North American Boreal', 'North American Temperate', 'South American Tropical'])

What if construct a pathological $\mathbf{S}_0$ which is extremely tight over land but very loose over the ocean?

In [ ]:
var4d = Var4D_Components('explore_unc_balance', verbose=False, store_intermediate=True) # change verbose to False to see fancy progress bars and suppress prints
flux_corr_structure = {'temp_corr': 2.0} # 2-month temporal correlation, no horizontal correlation
prior_flux_unc_dict = {'prior_unc_source': 'nee', 'prior_unc_scale': {'land': 0.001, 'ocean': 50.0}}
var4d.var4d_setup(obs_to_assim=obs_assim_dict, corr_structure=flux_corr_structure, **prior_flux_unc_dict)
var4d.var4d_chain(gradnorm=1.0E-5)

In [ ]:
po4 = Visualize_Obs('explore_unc_balance')
po4.plot_site(['spo','mlo','smo', 'brw'])

In [ ]:
vf4 = Visualize_Fluxes('explore_unc_balance')
vf4.plot_region(['Globe', 'Pacific Ocean', 'North American Temperate', 'South American Tropical'])

# Different ways of constructing $\mathbf{S_x}$
We have been setting the NEE error to be a factor $\times$ NEE. As discussed in the class, that's not always the best choice, especially in shoulder seasons or over regions where a small net flux is the result of large gross fluxes (such as the Tropics). Let's look at the base case, NEE uncertainty = 0.25 $\times$ NEE, and a "problem" region.

In [ ]:
vf1 = Visualize_Fluxes('only_noaa_observatories')
vf1.plot_region(['North American Boreal', 'South American Tropical', 'Tropical Asia', 'Tropical Africa'])

We did pretty good in the boreal region, but not so good in the Tropics. Let's try setting the uncertainty with ecosystem respiration.

In [ ]:
var4d = Var4D_Components('explore_nee_unc_source', verbose=False, store_intermediate=True) # change verbose to True to see the progress of the optimizer
flux_corr_structure = {'temp_corr': 2.0} # 2-month temporal correlation, no horizontal correlation
obs_assim_dict = {'sites': ['mlo', 'spo', 'brw', 'smo']} # just the four observatories
prior_flux_unc_dict = {'prior_unc_source': 'reco', 'prior_unc_scale': {'land': 0.25, 'ocean': 0.5}}
var4d.var4d_setup(obs_to_assim=obs_assim_dict, corr_structure=flux_corr_structure, **prior_flux_unc_dict)
var4d.var4d_chain(gradnorm=1.0E-5)

In [ ]:
vf2 = Visualize_Fluxes('explore_nee_unc_source')
vf2.plot_region(['North American Boreal', 'South American Tropical', 'Tropical Asia', 'Tropical Africa'])

We did **much** better over the Tropical regions! Can we see this "improvement" in obs space? Let's plot the fits w.r.t. sites we *did not assimilate*.

In [ ]:
po1 = Visualize_Obs('only_noaa_observatories')
po1.plot_site(['kum','amt','lef', 'crz'])

In [ ]:
po2 = Visualize_Obs('explore_nee_unc_source')
po2.plot_site(['kum','amt','lef', 'crz'])

Our fits to stuff we did not assimilate improved! So this is probably a **better** inversion.